# Библиотеки

In [30]:
from zlib import crc32

import numpy as np
import pandas as pd
import scipy.stats as st
from scipy.stats import jarque_bera
from scipy.stats import shapiro

import warnings
warnings.filterwarnings("ignore")

# Код генерации выборки

In [2]:
import re
EMAIL_REGEX = re.compile(r"[^@]+@phystech.edu")

def generate_dataset(code):
    rs = np.random.RandomState(code)
    data = rs.randn(100)
    noise_case = rs.randint(low = 0, high = 3)
    max_attemp_num = 10000
    if noise_case == 0:
        bad = True
        attemps = 0
        while bad and attemps < max_attemp_num:
            data2 = data + rs.laplace(size = (100)) * 0.25
            p1 = jarque_bera(data2)[1]
            p2 = shapiro(data2)[1]

            if p1 < 0.05 and p2 > 0.05:
                bad = False
        data = data2
        if bad:
            print ('could not')
    elif noise_case == 1:
        data = data + rs.gamma(1, size=(100))*0.5
    elif noise_case == 2:
        data = data +  rs.uniform()*1.5

    return data

# Генерация выборки для вашей почты

<span style="color:red">
    ВАЖНО!
    Почта, которую укажете ниже и почта с которой Вы отправляете домашнее задание должна совпадать!
    В момент проверки задания алгоритм генерирует выборку на основе почты из анкеты!
</span>

Внимательно проверьте почту для которой выполняется задание!

In [3]:
task = dict()
task['mail'] = input(prompt='Enter your mail: ')
assert EMAIL_REGEX.match(task['mail']), 'Not a mail in the phystech.edu domain'
task['id'] = crc32(task['mail'].encode('utf-8'))
task['data'] = generate_dataset(task['id'])

task

Enter your mail: egorov.sa@phystech.edu


{'mail': 'egorov.sa@phystech.edu',
 'id': 1581189463,
 'data': array([ 1.06037604, -0.15375779, -0.54514438, -0.25665905,  0.34882239,
        -0.2065498 ,  1.0977466 , -0.31542487,  0.20623395,  0.55553099,
        -2.70129363, -0.36156498, -0.03478419,  0.28278954,  0.36017102,
        -0.55951976, -0.59846063,  1.15594758, -0.38676524, -0.4706392 ,
        -1.3888606 ,  1.02212967,  1.30901239,  1.406303  ,  0.36458551,
        -1.48581811, -0.32734353,  0.5293529 , -0.75427568,  0.93831996,
         1.45094896, -2.27539463,  1.49666248, -0.9567224 , -0.37192089,
        -0.93075537, -0.80560881,  0.91729426, -0.36160874, -0.8402609 ,
         0.07588868, -0.44127798, -1.55640434,  0.89125963, -0.30225815,
        -0.66268948, -0.84859751, -0.12304585,  0.73437766,  0.91146308,
         0.56636882,  0.25427588, -0.1082083 ,  0.54025753,  0.21052126,
         1.64898193, -0.27364669, -0.80565764, -0.50084254, -0.30956891,
        -1.00416408, -2.27965641,  0.27841732,  0.58041744,  0

# Работа с выборкой

In [4]:
data = pd.DataFrame(task['data'])

## Выборочное среднее по выборке

In [5]:
print(f'mean = {data[0].mean():.3f}')

mean = -0.038


## Оценка стандартного отклонения (несмещенная)

In [8]:
print(f'unbiased std = {data[0].std(ddof=1):.3f}')

unbiased std = 0.991


## Можно ли отклонить гипотезу о нормальности выборки, используя критерий Харке-Бера (p<=0.05)?

In [16]:
st.jarque_bera(data[0])[1] <= 0.05

True

## Можно ли отклонить гипотезу о нормальности выборки, используя критерий Шапиро-Уилка (p<=0.05)?

In [19]:
st.shapiro(data[0])[1] <= 0.05

False

## Сумма единиц в бинаризованной выборке

In [20]:
(data[0] >= 0).astype(int).sum()

46

## Значение p-value для биномиального теста с двусторонней альтернативой (p0 = 0.5)

In [31]:
print(st.binom_test(((data[0] >= 0).astype(int).sum(), len(data[0])), p=0.5, alternative='two-sided'))

9.236846495694579e-06
